 # Benchmarking

In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
from benchmark import benchmark_csv
from benchmark.process import runnable, run
from os import PathLike, chdir
from pathlib import Path
from benchmark.api import Api, ApiBefore, ApiAfter
from time import sleep

if not "PROJECT_DIR" in globals():
    PROJECT_DIR = Path().resolve().parent
    chdir(PROJECT_DIR)


def out(name: str) -> PathLike:
    return PROJECT_DIR / "out" / f"{name}.csv"

## Clean up

Making sure nothing is running before the benchmarks.

In [8]:
run("make remove-before")
run("make remove-after")
run("make remove-queue")

CompletedProcess(args='make remove-queue', returncode=0)

## Compose Benchmarks

In [9]:
states = ["before", "after", "queue"]

### Cold Start

In [10]:
for state in states:
    benchmark_csv(
        out(f"cold-start_{state}"),
        func=runnable(f"make start-{state}"),
        after=runnable(f"make remove-{state}"),
    )

### Warm Start

In [11]:
for state in states:
    benchmark_csv(
        out(f"warm-start_{state}"),
        func=runnable(f"make start-{state}"),
        beforeAll=runnable(f"make start-{state} && make stop-{state}"),
        after=runnable(f"make stop-{state}"),
        afterAll=runnable(f"make remove-{state}"),
    )

### Stop

In [12]:
for state in states:
    benchmark_csv(
        out(f"stop_{state}"),
        before=runnable(f"make start-{state}"),
        func=runnable(f"make stop-{state}"),
        afterAll=runnable(f"make remove-{state}"),
    )

### Remove

In [13]:
for state in states:
    benchmark_csv(
        out(f"remove_{state}"),
        before=runnable(f"make start-{state}"),
        func=runnable(f"make remove-{state}"),
    )

## API

In [14]:
Api.create_user("albert", "einstein", "albert.einstein@example.com")

TypeError: Api.create_user() missing 1 required positional argument: 'email'

In [ ]:
api = ApiBefore("albert", "einstein")

In [ ]:
database_id = api.create_database("test")

In [ ]:
table_id = api.create_table(database_id, "test")

In [ ]:
api.insert_row(database_id, table_id, {})

In [ ]:
api.update_row(database_id, table_id, {"id": 1}, {"id": 11})

In [ ]:
api.delete_row(database_id, table_id, {"id": 11})

In [ ]:
api.import_csv(database_id, table_id, PROJECT_DIR / "test.csv")

In [ ]:
query_result = api.create_query(database_id, "SELECT id FROM test")
print(query_result)

{'result': [{'id': 4}, {'id': 5}, {'id': 6}], 'id': 4, 'result_number': 3}


In [ ]:
print(api.execute_query(database_id, query_result["id"]))

{'result': [{'id': 4}, {'id': 5}, {'id': 6}], 'id': 4, 'result_number': 3}


In [ ]:
apis: dict[str, Api] = {"before": ApiBefore(), "after": ApiAfter()}

### Create Database

In [ ]:
for state, api in apis.items():

    def beforeAll():
        run(f"make start-{state}")
        sleep(1)
        api.create_user("albert", "einstein", "albert.einstein@example.com")
        api.login("albert", "einstein")

    benchmark_csv(
        out(f"create-database_{state}"),
        beforeAll=beforeAll,
        func=lambda index: api.create_database(f"Test {index+1}"),
        afterAll=runnable(f"make remove-{state}"),
    )

### Create Database

In [ ]:
for state, api in apis.items():

    def beforeAll():
        run(f"make start-{state}")
        sleep(1)
        api.create_user("albert", "einstein", "albert.einstein@example.com")
        api.login("albert", "einstein")

    benchmark_csv(
        out(f"create-database_{state}"),
        beforeAll=beforeAll,
        func=lambda index: api.create_database(f"Test {index+1}"),
        afterAll=runnable(f"make remove-{state}"),
    )